In [1]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import load_model
import cv2

In [2]:
train = r"C:\Users\LIYANA\Downloads\emotion-detection-fer\train"
test = r"C:\Users\LIYANA\Downloads\emotion-detection-fer\test"

In [3]:
def load_images(folder):
    img = []
    label = []
    for sub in os.listdir(folder):
        subpath = os.path.join(folder, sub)
        if os.path.isdir(subpath):
            lab = sub
            for name in os.listdir(subpath):
                imgpath = os.path.join(subpath, name)
                im = cv2.imread(imgpath)

                if im is not None:
                    img.append(im)
                    label.append(lab)

    return img, label

In [4]:
train_img, train_lab = load_images(train)
train_img = np.array(train_img) / 255.0
test_img, test_lab = load_images(test)
test_img = np.array(test_img) / 255.0

In [5]:
label_encoder = LabelEncoder()
train_lab = label_encoder.fit_transform(train_lab)
train_lab = to_categorical(train_lab, num_classes=7)
test_lab = label_encoder.transform(test_lab)
test_lab = to_categorical(test_lab, num_classes=7)

In [6]:
X_train, X_val, y_train, y_val = train_test_split(train_img, train_lab, test_size=0.2, random_state=42)

In [7]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 3)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Flatten(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(7, activation='softmax')
])

In [8]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [9]:
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, mode='min')

In [11]:
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_val, y_val), callbacks=[checkpoint, early_stopping])

Epoch 1/50
359/359 [==============================] - 104s 288ms/step - loss: 1.5166 - accuracy: 0.4282 - val_loss: 1.4562 - val_accuracy: 0.4404
Epoch 2/50
359/359 [==============================] - 96s 269ms/step - loss: 1.3947 - accuracy: 0.4704 - val_loss: 1.3489 - val_accuracy: 0.4850
Epoch 3/50
359/359 [==============================] - 97s 270ms/step - loss: 1.3285 - accuracy: 0.4936 - val_loss: 1.3011 - val_accuracy: 0.5080
Epoch 4/50
359/359 [==============================] - 95s 264ms/step - loss: 1.2794 - accuracy: 0.5122 - val_loss: 1.4513 - val_accuracy: 0.4596
Epoch 5/50
359/359 [==============================] - 96s 266ms/step - loss: 1.2412 - accuracy: 0.5277 - val_loss: 1.2366 - val_accuracy: 0.5387
Epoch 6/50
359/359 [==============================] - 96s 268ms/step - loss: 1.2010 - accuracy: 0.5419 - val_loss: 1.2510 - val_accuracy: 0.5347
Epoch 7/50
359/359 [==============================] - 108s 301ms/step - loss: 1.1709 - accuracy: 0.5565 - val_loss: 1.1889 - val_

In [12]:
best_model = load_model('best_model.h5')

In [14]:
import shutil

source = 'best_model.h5'
destination = r"C:\Users\LIYANA\OneDrive\Documents\Projects\best_model_copy.h5"  # Different filename

shutil.copyfile(source, destination)
print(f'Saved model to {destination}')

Saved model to C:\Users\LIYANA\OneDrive\Documents\Projects\best_model_copy.h5


In [ ]:
val_loss, val_accuracy = best_model.evaluate(X_val, y_val)
print(f'Validation Loss: {val_loss}')
print(f'Validation Accuracy: {val_accuracy}')

180/180 [==============================] - 7s 33ms/step - loss: 1.1112 - accuracy: 0.6181
Validation Loss: 1.1112223863601685
Validation Accuracy: 0.6180773377418518


In [ ]:
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [ ]:
def detect_emotion(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_gray = cv2.resize(roi_gray, (48, 48), interpolation=cv2.INTER_AREA)
        
        if np.sum([roi_gray]) != 0:
            roi = roi_gray.astype('float32') / 255.0
            roi = np.expand_dims(roi, axis=0)
            
            prediction = best_model.predict(roi)[0]
            max_index = int(np.argmax(prediction))
            emotion = emotion_labels[max_index]
            
            label_position = (x, y-10)
            cv2.putText(frame, emotion, label_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        else:
            cv2.putText(frame, 'No Face Found', (20, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    return frame

In [27]:
from tensorflow.keras.preprocessing.image import img_to_array
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    frame = detect_emotion(frame)
    cv2.imshow('Emotion Detector', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

ValueError: in user code:

    File "C:\Users\LIYANA\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py", line 2341, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\LIYANA\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py", line 2327, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\LIYANA\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py", line 2315, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\LIYANA\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py", line 2283, in predict_step
        return self(x, training=False)
    File "C:\Users\LIYANA\AppData\Roaming\Python\Python311\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\LIYANA\AppData\Roaming\Python